In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from powersimdata.input.grid import Grid

In [2]:
county_list_owner = pd.read_excel('SPP_Counties_Owners.xlsx',index_col = 0)

In [3]:
legacy_ba_to_owners = {
    'CSWS':{'AEP','AECC','SPA'}, # AECC and SPA are approximately merged into CSWS
    'EDE':{'EDE'},
    'MPS':{'GMO'},
    'GRDA':{'GRDA'},
    'KACY':{'KACY'},
    'KCPL_INDN':{'KCPL'},
    'LES':{'LES'},
    'NPPD':{'NPPD'},
    'OKGE':{'OGE'},
    'OPPD':{'OPPD'},
    'SPRM':{'SPRM'},
    'SPS':{'SPS'},
    'SECI':{'SUNC','MIDW','MKEC'}, # MIDW and MKEC are approximately merged into SECI
    'WAUE':{'UMZ'},
    'WR':{'WESTAR'},
    "WFEC":{'WFEC'},
}
# legacy BA INDN has hourly load profile, but the territory definition is missing. We merge the load of INDN into KCPL at this point.

In [4]:
owner_county_list = defaultdict(set)
for index,row in county_list_owner.iterrows():
    owner_county_list[row['OWNER']].add(''.join(row['COUNTY'].split()[:-1])+'__'+row['STATE'])

In [5]:
legacy_ba_county_list = defaultdict(set)
for legacy_ba,owners in legacy_ba_to_owners.items():
    for owner in owners:
        legacy_ba_county_list[legacy_ba] |= owner_county_list[owner]

In [6]:
bus_ba_map = pd.read_csv('../bus_ba_map.csv',index_col = 0)
bus_ba_map_spp = bus_ba_map[bus_ba_map['BA']=='SWPP']

In [7]:
for index,row in bus_ba_map_spp.iterrows():
    for legacy_ba, county_list in legacy_ba_county_list.items():
        if row['County'] in county_list:
            bus_ba_map_spp.loc[index,'BA'] = legacy_ba

C:\Users\bxia\Anaconda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [8]:
bus_no_match = bus_ba_map_spp[bus_ba_map_spp['BA']=='SWPP']

In [9]:
# Manual fix the buses that locate in the counties not in the legacy county list
for index,row in bus_no_match.iterrows():
    if row['County'][-2:] in {'MT','ND','SD','WY'}:
        bus_ba_map_spp.loc[index,'BA'] = 'WAUE'
    if row['County'][-2:] in {'NE'}:
        bus_ba_map_spp.loc[index,'BA'] = 'NPPD'
    if row['County'][-2:] in {'NM'}:
        bus_ba_map_spp.loc[index,'BA'] = 'SPS'
    if row['County'][-2:] in {'OK','LA','TX','AR'}:
        bus_ba_map_spp.loc[index,'BA'] = 'CSWS'
    if row['zone_name'] in {'Missouri West','Kansas','Iowa'}:
        bus_ba_map_spp.loc[index,'BA'] = 'KCPL_INDN'
    if row['County'][-2:] in {'VA'}:
        bus_ba_map_spp.loc[index,'BA'] = 'PJM'
# The rest of the buses belong to another ISO, AECI, which previously merged into MISO
bus_ba_map_spp.replace('SWPP','AECI',inplace=True)

C:\Users\bxia\Anaconda\lib\site-packages\pandas\core\frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [10]:
bus_no_match = bus_ba_map_spp[bus_ba_map_spp['BA']=='SWPP']
bus_no_match

,Pd,lat,lon,County,BA,zone_name
bus_id,,,,,,


In [20]:
bus_ba_map_spp.to_csv('bus_ba_map_spp.csv')

### Sanity check

In [11]:
#Plot on the counties of a sub area on the map
from bokeh.io import show,output_notebook,output_file,export
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper,
    CustomJS,
    Slider,
    CheckboxGroup,
    Select,
    Div,
    TapTool,
    ColorBar,
    LinearColorMapper
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
from bokeh.layouts import (widgetbox, row, gridplot)

# from bokeh.tile_providers import CARTODBPOSITRON,STAMEN_TONER_BACKGROUND,STAMEN_TONER,STAMEN_TERRAIN

# import bokeh.sampledata
# bokeh.sampledata.download()
from bokeh.sampledata.us_counties import data as us_counties
from bokeh.sampledata.us_states import data as us_states
from bokeh import events

import matplotlib.pyplot as plt
from bokeh.colors.groups import yellow
import bokeh.colors

import seaborn as sns
import matplotlib.cm as cm
import matplotlib

In [12]:
state_xs = [us_states[state]["lons"] for state in us_states if state not in {'HI','AK'}]
state_ys = [us_states[state]["lats"] for state in us_states if state not in {'HI','AK'}]

In [13]:
area_label = {}
cnt = 0
for area in legacy_ba_to_owners:
    cnt += 1
    area_label[area] = cnt
legacy_ba_county_list_plot = {}
for index,row in bus_ba_map_spp.iterrows():
    if row['BA'] in area_label:
        legacy_ba_county_list_plot[row['County'][:-4]+'_'+row['County'][-2:].lower()] = (row['BA'],area_label[row['BA']])

In [14]:
counties_xs = []
counties_ys = []
counties_name = []
area_color = []
for code in us_counties:
    key = str(us_counties[code]['name'])+'_'+str(us_counties[code]['state'])
    if key in legacy_ba_county_list_plot:
        counties_xs.append(us_counties[code]['lons'])
        counties_ys.append(us_counties[code]['lats'])
        counties_name.append(key+':'+legacy_ba_county_list_plot[key][0])
        area_color.append(bokeh.palettes.Category20[20][(legacy_ba_county_list_plot[key][1]-1)%20])
source = ColumnDataSource(data=dict(
    x = counties_xs,
    y = counties_ys,
    name = counties_name,
    color = area_color
))

In [15]:
p = figure(title="BA County Check", 
           toolbar_location="left", plot_width=1300, plot_height=700,
           tools = ['tap','box_zoom','wheel_zoom','save','reset'])

p.patches(state_xs, state_ys, fill_alpha=0,
    line_color="gray", line_alpha=0.5, line_width=1.0)

county_render = p.patches('x','y', source = source, fill_alpha=0.5,
    line_color="gray", line_alpha=0.5, line_width=0.5, color = 'color')

hover1 = HoverTool(renderers=[county_render],name='Test')
hover1.point_policy = "follow_mouse"
hover1.tooltips = [
    ("County", "@name")
]
p.add_tools(hover1) 
show(p)

In [16]:
eastern = Grid(['Eastern'])

--> Loading Eastern interconnect
Loading zone
Loading sub
Loading bus2sub
Loading bus
Loading plant
Loading plant cost
Loading branch
Loading DC line


In [17]:
# Plot for buses assigned to a specific BA area
source = eastern.bus.loc[list(bus_ba_map_spp[bus_ba_map_spp['BA'].isin(legacy_ba_to_owners)].index)]
source.loc[:,'area_color'] = source['zone_id'].apply(lambda x: bokeh.palettes.Category20[20][(x-1)%20])

p = figure(title="", 
           toolbar_location="left", plot_width=1300, plot_height=700)

p.patches(state_xs, state_ys, fill_alpha=0,
    line_color="gray", line_alpha=0.5, line_width=1.0)
p.patches(counties_xs, counties_ys, fill_alpha=0,
    line_color="gray", line_alpha=0.5, line_width=0.5)

p.scatter(source=source,x='lon',y='lat',size=2,marker='o',color='area_color')

show(p)